In [ ]:
import time

import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os

In [ ]:
def exponential_smoothing(alpha, s):
    '''
    Primary exponential smoothing
    :param alpha:  Smoothing factor,num
    :param s:      List of data,list
    :return:       List of data after smoothing,list
    '''

    if not s:
        return []

    s_temp = [s[0]]
    print(s_temp)
    for i in range(1, len(s), 1):
        s_temp.append(alpha * s[i - 1] + (1 - alpha) * s_temp[i - 1])
    return s_temp

In [ ]:
def precess_image(image):
    '''
    Graying and GaussianBlur
    :param image: The image matrix,np.array
    :return: The processed image matrix,np.array
    '''
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  #Grayscale
    gray_image = cv2.GaussianBlur(gray_image, (3, 3), 0)  #Gaussian filtering
    return gray_image

In [ ]:
def abs_diff(pre_image, curr_image):
    '''
    Calculate absolute difference between pre_image and curr_image
    :param pre_image:The image in past frame,np.array
    :param curr_image:The image in current frame,np.array
    :return:
    '''
    gray_pre_image = precess_image(pre_image)
    gray_curr_image = precess_image(curr_image)
    diff = cv2.absdiff(gray_pre_image, gray_curr_image)
    res, diff = cv2.threshold(diff, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    #  fixme：Here first write it in the form of a simple sum
    cnt_diff = np.sum(np.sum(diff))
    return cnt_diff

In [ ]:
class KeyFrameGetter:
    '''
    Get the key frame
    '''

    def __init__(self, video_path, img_path, video, window=25):
        '''
        Define the param in model.
        :param video_path: The path points to the movie data,str
        :param img_path: The path we save the image,str
        :param window: The comparing domain which decide how wide the peek serve.
        '''
        self.window = window  # Number of key frames
        self.video_path = video_path  # Video path
        self.img_path = img_path  #Image storage path
        self.diff = []
        self.idx = []
        self.video = video
        
    def load_diff_between_frm(self, smooth=True, alpha=0.07):
        '''
        Calculate and get the model param
        :param smooth: Decide if you want to smooth the difference.
        :param alpha: Difference factor
        :return:
        '''
        print("load_diff_between_frm")
        cap = cv2.VideoCapture(self.video_path)  # Open video file
        diff = []
        frm = 0
        pre_image = np.array([])
        curr_image = np.array([])

        while True:
            frm = frm + 1

            # success Indicates whether it is successful, data is the image data of the current frame; 
            #.read reads one frame of image and moves to the next frame
           # cv2.imwrite('./img/' + str(num) + ".jpg", data)
            success, data = cap.read()
            if not success:
                break
            #  # Here is the main body
            #    # Record the data each time
            #  TODO: It can be optimized here, for example, one batch can be entered to speed up the rotation speed
            if frm == 1:
                pre_image = data
                curr_image = data
            else:
                pre_image = curr_image
                curr_image = data
            #  Make difference
            diff.append(abs_diff(pre_image, curr_image))
            #  End Loop

            if frm % 100 == 0:
                print('Detect Frame:', str(frm))
        cap.release()

        if smooth:
            diff = exponential_smoothing(alpha, diff)
        #  Standardized data
        self.diff = np.array(diff)
        mean = np.mean(self.diff)
        dev = np.std(self.diff)
        self.diff = (self.diff - mean) / dev

        #  Standardized data
        self.pick_idx()
        return

    def pick_idx(self):
        '''
        Get the index which accord to the frame we want(peek in the window)
        :return:
        '''
        print("pick_idx")
        for i, d in enumerate(self.diff):
            ub = len(self.diff) - 1
            lb = 0
            if not i - self.window // 2 < lb:
                lb = i - self.window // 2
            if not i + self.window // 2 > ub:
                ub = i + self.window // 2

            comp_window = self.diff[lb:ub]
            if d >= max(comp_window):
                self.idx.append(i)

        tmp = np.array(self.idx)
        tmp = tmp + 1  # to make up the gap when diff
        self.idx = tmp.tolist()
        print("Extract the Frame Index:" + str(self.idx))

    def save_key_frame(self):
        '''
        Save the key frame image
        :return:
        '''
        print("save_key_frame")
        cap = cv2.VideoCapture(self.video_path)  # Open video file
        frm = 0
        idx = set(self.idx)
        while True:
            frm = frm + 1
            success, data = cap.read()
            if not success:
                break
            if frm in idx:
                print('Extracting idx:'+str(frm))
                cv2.imwrite(self.img_path + '/' +str(frm) + "-" + self.video + ".jpg", data)
                idx.remove(frm)
            if not idx:
                print('Done！')
                break
        return self.idx

    def plot_diff_time(self):
        '''
        Plot the distribution of the difference along to the frame increasing.
        :return:
        '''
        plt.plot(self.diff, '-b')
        plt.plot(np.array(self.idx) - 1, [self.diff[i-1] for i in self.idx], 'or')
        plt.xlabel('Frame Pair Index')
        plt.ylabel('Difference')
        plt.legend(['Each Frame', 'Extract Frame'])
        plt.title("The Difference for Each Pair of Frame")
        plt.plot()
        plt.show()

    

In [ ]:
def image_generation(root, video, frames, destination, keyframes, kernel_size=17):
    os.makedirs(destination, exist_ok=True)
    image_filenames = os.listdir(os.path.join(keyframes, video))
    sorted_filenames = sorted(image_filenames, key=lambda x: int(x.split('.')[0]))
    for image in frames:
        index = image
        if index < len(sorted_filenames) - kernel_size - 1:
            energy_image = 0.0
            for image in range(index, index + kernel_size):
                print(image)
                im = Image.open(os.path.join(keyframes, video, sorted_filenames[image]))
                image_array = np.array(im)
                energy_image += image_array/kernel_size
            energy_image = energy_image.astype(np.uint8)
            save_image = Image.fromarray(energy_image)
            save_image.save(f"{destination}/{video}-{image}.jpg")

In [ ]:
if __name__ == '__main__':
    sa = []
    for root, dirs, files in os.walk('/Users/aravdhoot/PD/masked_videos/MODERATE-20230609T012809Z-001/MODERATE'):
        for file in files:
            print(file)

            source_path = os.path.join(root, file)
            #+ file.strip('.avi').strip('.mp4') +
            dir_path = '/Users/aravdhoot/PD/keyframes/MODERATE'
            frame_path = '/Users/aravdhoot/PD/frames_updated/MODERATE'

            print("The source file directory is", root)
            print("The source file path is", source_path.split('/')[-1].split()[0])
            print("The destination file path is", dir_path)

            if not os.path.exists(dir_path):
                os.mkdir(dir_path)

            kfg = KeyFrameGetter(source_path, dir_path, source_path.split('/')[-1].split()[0],  20)

            
            a = time.time()
            kfg.load_diff_between_frm(alpha=0.07)  # GetTake model parameters
            b = time.time()
            sa.append(b - a)
            print(sa)
            frames = kfg.save_key_frame()

            image_generation(frame_path, source_path.split('/')[-1].split()[0], frames, destination='/Users/aravdhoot/Parkinson-Project/keyframe_energy/MODERATE', keyframes='/Users/aravdhoot/Parkinson-Project/hpe_images/MODERATE')

            kfg.plot_diff_time()